In [ ]:
# Import required libraries
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import boto3
import json

# Initialize SageMaker session and role
session = sagemaker.Session()
role = sagemaker.get_execution_role()

def deploy_financial_model():
    # Model configuration
    model_config = {
        'transformers_version': '4.26',
        'pytorch_version': '1.13',
        'python_version': 'py39',
        'model_id': 'ProsusAI/finbert',  # Financial BERT model
        'instance_type': 'ml.g5.xlarge',
        'instance_count': 1
    }
    
    # Environment variables for the model
    environment = {
        'HF_MODEL_ID': model_config['model_id'],
        'HF_TASK': 'text-classification',
        'MAX_LENGTH': '512',
        'BATCH_SIZE': '32'
    }
    
    # Create HuggingFace Model
    huggingface_model = HuggingFaceModel(
        transformers_version=model_config['transformers_version'],
        pytorch_version=model_config['pytorch_version'],
        py_version=model_config['python_version'],
        model_id=model_config['model_id'],
        role=role,
        env=environment
    )
    
    # Deploy model
    print(f"Deploying model {model_config['model_id']}...")
    predictor = huggingface_model.deploy(
        initial_instance_count=model_config['instance_count'],
        instance_type=model_config['instance_type'],
        endpoint_name='financial-analysis-endpoint'
    )
    
    return predictor

def test_model(predictor):
    # Test data
    test_text = """
    Apple Inc. reported strong fourth-quarter earnings, 
    beating analyst expectations with revenue up 20% year-over-year. 
    The company's services segment showed particularly strong growth.
    """
    
    # Make prediction
    response = predictor.predict({
        'inputs': test_text,
        'parameters': {
            'return_all_scores': True
        }
    })
    
    print("\nTest Results:")
    print(f"Input text: {test_text}")
    print(f"Model prediction: {json.dumps(response, indent=2)}")
    
    return response

def cleanup_endpoint():
    """Clean up the endpoint to avoid unnecessary charges"""
    sagemaker_client = boto3.client('sagemaker')
    try:
        sagemaker_client.delete_endpoint(EndpointName='financial-analysis-endpoint')
        print("Endpoint deleted successfully")
    except Exception as e:
        print(f"Error deleting endpoint: {str(e)}")

# Main execution
if __name__ == "__main__":
    try:
        # Deploy model
        predictor = deploy_financial_model()
        
        # Test the deployed model
        test_results = test_model(predictor)
        
        print("\nModel deployment and testing completed successfully!")
        
        # Uncomment the following line to clean up the endpoint
        # cleanup_endpoint()
        
    except Exception as e:
        print(f"Error during model deployment or testing: {str(e)}")
        cleanup_endpoint()

In [ ]:
# Get endpoint cost estimation
instance_type = 'ml.g5.xlarge'
region = session.boto_session.region_name
pricing = boto3.client('pricing', region_name='us-east-1')

# Get pricing info
response = pricing.get_products(
    ServiceCode='AmazonSageMaker',
    Filters=[
        {'Type': 'TERM_MATCH', 'Field': 'instanceType', 'Value': instance_type},
        {'Type': 'TERM_MATCH', 'Field': 'location', 'Value': region}
    ]
)

print(f"Estimated hourly cost for {instance_type}: ", response)